# PROJETO ETL + BI - Análise do desempenho em Ciências da Natureza dos alunos do Ceará no ENEM 2022

## Primeira parte: EXTRAÇÃO DE DADOS:

In [4]:
#Instalação da bilbioteca dask para gerenciamento de datframe de grandes tamanhos. Será Utilizada ao invés do Pandas 
#devido ao tamanho do arquivo .csv
%pip install dask
%pip install --upgrade pandas
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/b6/7c/8debebb4f90174074b827c63242c23851bdf00a532489fba57fef3416e40/charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for idna<4,>=2.5 from https://files.pythonhosted.org/packages/c2/e7/a82b05cf63a603df6e68d59ae6a68bf5064484a0718ea5033660af4b54a9/idna-3.6-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/88/75/311454fd3317aefe18415f04568edc20218453b709c63c58b9292c71be17/urllib3-2.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e913d6c962

### 1) - Importação de bibliotecas 

In [1]:
#importação de Bibliotecas
import dask.dataframe as dd
import pandas as pd
import zipfile
import requests
import os
from io import BytesIO

### 2)- Criação de pasta e Download do Arquivo do Site do Inep

In [2]:
#Criar estrutura de pastas para armazenamento do arquivo baixado
os.makedirs('./enem2022', exist_ok=True)

In [7]:
#Com o arquivo ZIP já baixado do site do INEP, podemos extraí-lo na pasta que criamos:
file = r'C:\Users\ycaro\OneDrive\Desktop\Imersao Dados\Projeto ETL + BI\enem2022\microdados_enem_2022.zip'
myzip = zipfile.ZipFile(file).extractall('./enem2022')
print('Processamento Concluído.')

Processamento Concluído.


### 3)- Lendo os dados em lote (Arquivo Muito Grande) e criando um dataframe para o estado do Ceará

In [8]:
# Cria dataframe com mais memória devido ao tamanho do arquivo
enem = dd.read_csv(
    './enem2022/DADOS/microdados_enem_2022.csv',
    sep = ';' , 
    decimal=',' ,
    dtype={'CO_MUNICIPIO_NASCIMENTO': 'float64',
            'CO_UF_NASCIMENTO': 'float64',
            'NU_IDADE': 'float64',
            'NO_MUNICIPIO_ESC': 'object',
            'SG_UF_ESC': 'object'},
    encoding='latin1'
)

In [10]:
#Recriando o Dataframe Só para Ceará = Processo de Filtragem
enem = enem.loc[enem.SG_UF_ESC == 'CE']

## Segunda parte: Transformando os dados

In [12]:
#Recriando o Dataframe só com as colunas necessárias
enem_ce = enem[['NU_INSCRICAO',         
                'SG_UF_ESC',     
                'TP_SEXO',
                'TP_COR_RACA',
                'TP_ESCOLA',
                'TP_ENSINO',
                'NU_NOTA_CN',
                'NU_NOTA_MT',
                'Q001',
                'Q002',
                'Q004',
                'Q005',
                'Q006']].compute()

In [13]:
#Analisando Quantidade de Nulos no Dataframe
enem_ce.isnull().sum()

NU_INSCRICAO        0
SG_UF_ESC           0
TP_SEXO             0
TP_COR_RACA         0
TP_ESCOLA           0
TP_ENSINO        3152
NU_NOTA_CN      31135
NU_NOTA_MT      31135
Q001                0
Q002                0
Q004                0
Q005                0
Q006                0
dtype: int64

In [14]:
#Cria backuo do DataFrame
Enem_bkp = enem_ce

In [15]:
#Apagando as Linhas com Notas Zeradas - Excluindos Alunos Faltantes
enem_ce = enem_ce.dropna(subset=['NU_NOTA_MT'])

In [16]:
#Quantidade de Linhas
enem_ce.shape[0]

71837

In [17]:
#Verificando os Tipos de Dados do Dataframe
enem_ce.dtypes

NU_INSCRICAO              int64
SG_UF_ESC       string[pyarrow]
TP_SEXO         string[pyarrow]
TP_COR_RACA               int64
TP_ESCOLA                 int64
TP_ENSINO               float64
NU_NOTA_CN      string[pyarrow]
NU_NOTA_MT      string[pyarrow]
Q001            string[pyarrow]
Q002            string[pyarrow]
Q004            string[pyarrow]
Q005                      int64
Q006            string[pyarrow]
dtype: object

In [18]:
#Atualizando Tipos de Dados de Colunas Importantes
enem_ce['NU_NOTA_CN'] = enem_ce['NU_NOTA_CN'].astype('float')
enem_ce['NU_NOTA_MT'] = enem_ce['NU_NOTA_MT'].astype('float')

C:\Users\ycaro\AppData\Local\Temp\ipykernel_8796\1672564464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enem_ce['NU_NOTA_CN'] = enem_ce['NU_NOTA_CN'].astype('float')
C:\Users\ycaro\AppData\Local\Temp\ipykernel_8796\1672564464.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enem_ce['NU_NOTA_MT'] = enem_ce['NU_NOTA_MT'].astype('float')


In [20]:
#Analise detalhada do Dataframe
enem_ce.describe()

,NU_INSCRICAO,TP_COR_RACA,TP_ESCOLA,TP_ENSINO,NU_NOTA_CN,NU_NOTA_MT,Q005
count,7.183700e+04,71837.000000,71837.000000,71030.000000,71837.000000,71837.000000,71837.000000
mean,2.100563e+11,2.394268,2.095327,1.003111,474.308997,507.794323,4.015814
std,1.000584e+06,0.971131,0.293668,0.055693,64.816970,109.061420,1.279771
min,2.100545e+11,0.000000,2.000000,1.000000,0.000000,0.000000,1.000000
25%,2.100553e+11,1.000000,2.000000,1.000000,424.700000,421.500000,3.000000
50%,2.100563e+11,3.000000,2.000000,1.000000,463.000000,485.800000,4.000000
75%,2.100571e+11,3.000000,2.000000,1.000000,514.800000,579.200000,5.000000
max,2.100580e+11,5.000000,3.000000,2.000000,825.000000,975.700000,20.000000


### Retirando alguns Insights

In [21]:
#Percentual de Participação por Sexo Declarado o Enem no Ceará
round((enem_ce.TP_SEXO.value_counts() / enem_ce.shape[0] ) * 100,2)

TP_SEXO
F    52.91
M    47.09
Name: count, dtype: double[pyarrow]

In [22]:
#Média geral de Ciências da Natureza no Ceará
print('Média Geral de Ciências da Natureza: ', round(enem_ce['NU_NOTA_CN'].mean(),2))

Média Geral de Ciências da Natureza:  474.31


In [23]:
#Média geral de Ciências da Natureza nas Escolas Particulares 
print('Ciências da Natureza (Particular): ', round(enem_ce['NU_NOTA_CN'].loc[enem_ce.TP_ESCOLA == 3].mean(),2))

Ciências da Natureza (Particular):  542.84


In [24]:
#Média geral de Matemática dos alunos de Escola Pública 
print('Ciências da Natureza (Pública): ', round(enem_ce['NU_NOTA_CN'].loc[enem_ce.TP_ESCOLA == 2].mean(),2))

Ciências da Natureza (Pública):  467.09


In [25]:
#Comparação de Médias de Matemática por raça declarada
print('Ciências da Natureza (Não Declarados): ', round(enem_ce['NU_NOTA_CN'].loc[enem_ce.TP_COR_RACA == 0].mean(),2))
print('Ciências da Natureza (Branco)        : ', round(enem_ce['NU_NOTA_CN'].loc[enem_ce.TP_COR_RACA == 1].mean(),2))
print('Ciências da Natureza (Preto)         : ', round(enem_ce['NU_NOTA_CN'].loc[enem_ce.TP_COR_RACA == 2].mean(),2))
print('Ciências da Natureza (Pardo)         : ', round(enem_ce['NU_NOTA_CN'].loc[enem_ce.TP_COR_RACA == 3].mean(),2))
print('Ciências da Natureza (Amarelo)       : ', round(enem_ce['NU_NOTA_CN'].loc[enem_ce.TP_COR_RACA == 4].mean(),2))
print('Ciências da Natureza (Indígena)      : ', round(enem_ce['NU_NOTA_CN'].loc[enem_ce.TP_COR_RACA == 5].mean(),2))

Ciências da Natureza (Não Declarados):  461.38
Ciências da Natureza (Branco)        :  498.81
Ciências da Natureza (Preto)         :  467.11
Ciências da Natureza (Pardo)         :  466.86
Ciências da Natureza (Amarelo)       :  472.43
Ciências da Natureza (Indígena)      :  456.8


In [26]:
#Comparação de Médias de Matemática por Renda declarada
print('Média Ciências da Natureza (Sem Renda)                      : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'A'].mean(),2))
print('Média Ciências da Natureza (Até R$ 998,00)                  : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'B'].mean(),2))
print('Média Ciências da Natureza (De R$ 2.495,01 até R$ 2.994,00) : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'F'].mean(),2))
print('Média Ciências da Natureza (De R$ 2.994,01 até R$ 3.992,00) : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'G'].mean(),2))  
print('Média Ciências da Natureza (De R$ 5.988,01 até R$ 6.986,00) : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'J'].mean(),2))
print('Média Ciências da Natureza (Mais de R$ 19.960,00)           : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'Q'].mean(),2))

Média Ciências da Natureza (Sem Renda)                      :  465.67
Média Ciências da Natureza (Até R$ 998,00)                  :  483.73
Média Ciências da Natureza (De R$ 2.495,01 até R$ 2.994,00) :  579.39
Média Ciências da Natureza (De R$ 2.994,01 até R$ 3.992,00) :  594.68
Média Ciências da Natureza (De R$ 5.988,01 até R$ 6.986,00) :  634.73
Média Ciências da Natureza (Mais de R$ 19.960,00)           :  697.87


## Terceira parte: Carga dos dados
### Realizando Carga de dados no SQlite 

In [27]:
#Importando biblioteca SQLite
import sqlite3 as lite

In [28]:
#Cria/Conceta Banco de Dados ENEM_2022.db e o cursor de navegação interna
con = lite.connect("ENEM_2022.db")
cur = con.cursor()

In [29]:
# Deletando Tabela com mesmo nome caso exista
cur.execute("DROP TABLE IF EXISTS Enem22")
con.commit()

In [31]:
# criando a tabela (Enem22)
cur.execute("""
CREATE TABLE IF NOT EXISTS Enem22 (
        NU_INSCRICAO INT,
        SG_UF_ESC VARCHAR(2),
        TP_SEXO VARCHAR(1),
        TP_COR_RACA INT,
        TP_ESCOLA INT,
        TP_ENSINO INT,
        NU_NOTA_CN FLOAT,
        NU_NOTA_MT FLOAT,
        Q001 VARCHAR(1),
        Q002 VARCHAR(1),
        Q004 VARCHAR(1),
        Q005 INT,
        Q006 VARCHAR(1) );
""")
con.commit()

In [32]:
#Carregando Dataframe Pandas na Tabela SQLite
cur.executemany("INSERT INTO Enem22 VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?);", enem_ce.values.tolist())
con.commit()

In [33]:
#Lendo os Dados do Banco de Dados
cur.execute("SELECT count(*)  FROM Enem22;")
for linha in cur.fetchall():
    print(f'Número de Registros: {linha[0]}')

Número de Registros: 71837


In [34]:
# Média Geral de Ciências da Natureza
cur.execute("SELECT avg(NU_NOTA_CN)  FROM Enem22;")
for linha in cur.fetchall():
    print(f'Nota Geral de Ciências da Natureza: {round(linha[0],2)}')

Nota Geral de Ciências da Natureza: 474.31


In [35]:
#Fecha a Conexão com o Banco de Dados
con.close()